In [58]:
from search_engine.search_engine import SearchEngine
import networkx as nx
from networkx import Graph
from sklearn.model_selection import train_test_split
import implicit
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import igraph
import warnings
import csv
warnings.filterwarnings('ignore')

In [3]:
search_engine = SearchEngine()

KeyboardInterrupt: 

In [4]:
search_engine.tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 20 columns):
created_at                 10001 non-null object
id_str                     10001 non-null object
text                       10001 non-null object
source                     10001 non-null object
truncated                  10001 non-null bool
in_reply_to_screen_name    1522 non-null object
user                       10001 non-null object
retweeted_status           6973 non-null object
reply_count                10001 non-null int64
retweet_count              10001 non-null int64
favorite_count             10001 non-null int64
entities                   10001 non-null object
favorited                  10001 non-null bool
retweeted                  10001 non-null bool
filter_level               10001 non-null object
timestamp_ms               10001 non-null object
extended_tweet             1451 non-null object
extended_entities          441 non-null object
withheld_in_countries  

In [ ]:
#u -> search_engine.tweets.user[0]['screen_name']
#v -> search_engine.tweets.retweeted_status[1]['user']['screen_name']

In [54]:
#Para Poner bonito -> dictionario de los usuarios v
#Mirar página en favoritos, lo del color también en la NS_P09
#Add weight edges from how many times user u retweeted user v

#g = nx.Graph()
#for tweet in search_engine.tweets.iterrows():
#    if str(tweet[1]['retweeted_status'])!='nan':
#        u=tweet[1]['user']['screen_name']
#        v=tweet[1]['retweeted_status']['user']['screen_name']
#        g.add_node(u)
#        g.add_node(v)
#        g.add_edge(u,v)

g=igraph.Graph()
for tweet in search_engine.tweets.iterrows():
    if str(tweet[1]['retweeted_status'])!='nan':
        u=tweet[1]['user']['screen_name']
        v=tweet[1]['retweeted_status']['user']['screen_name']
        g.add_vertices(u)
        g.add_vertices(v)
        g.add_edges([(u,v)])

In [61]:
#pos=nx.spring_layout(g)
#plt.figure(figsize=(15,15))
#nx.draw_networkx(g, pos, node_shape='o', node_size=50, with_labels=False, font_weight='bold')
#plt.show()

#layout = g.layout("kk")
#igraph.plot(g, layout = layout, target="/other-outputs/igraph.png")

In [7]:
#Recomender System
# -ALS -> No se que hacer con test

In [135]:
def find_nodes_at_distance_2(graph):

    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        
        # all the nodes at distance 1
        #nodes_at_most_distant_1 = set(graph.neighborhood(n1, order=1))

        # all the nodes at distance 1 and distance 2

        nodes_at_most_distant_2 = set(graph.neighborhood(n1, order=2))

        # only the nodes at distance 2
        #only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        
        # check if empty set
        if len(nodes_at_most_distant_2) > 0:
            for n2 in nodes_at_most_distant_2:
                
                # since n1 is an igraph vertex object, we need to extract the id
                n1_index = n1.index
                if n1_index!=n2:
                    all_potential_recommendations.add((n1_index, n2))
            
    return all_potential_recommendations

def predict_ALS(testset, model):
    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1,n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [136]:
edges_max_distance_2=find_nodes_at_distance_2(g)

# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(g.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N),replace=False)

ALS (Alternate Least Squares) #VER SI SON LOS FRIENDS-OF-FRIENDS

Version 3

In [161]:
def number_to_username(ALS_list: list, g) -> list:
    ALS_list_transformed=[]
    for i in range(len(ALS_list)):
        name=g.vs[ALS_list[i][0]]['name']
        new_tuple=(name, ALS_list[i][1])
        ALS_list_transformed.append(new_tuple)
    return ALS_list_transformed

def recommend_users(user_id: int, G:csr_matrix, g: igraph.Graph, top: int = 10) -> list:
    ALS_recommended_users=model.recommend(user_id, G, top)
    ALS_recommended_users_with_names=number_to_username(ALS_recommended_users, g)
    return ALS_recommended_users_with_names

In [130]:
ground_truth = set()
trainset = set()
for idx, one_edge in enumerate(g.es):
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target
    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
    else:
        trainset.add((n1, n2, 1))
        
for rec in edges_max_distance_2:
    
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    
    ground_truth.add((n1,n2,0))

In [131]:
G = g.get_adjacency().data
G = csr_matrix(G)

In [143]:
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=5, calculate_training_loss=True)
model.fit(G)

In [144]:
# generate the predictions
df_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])

all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: round(x))

In [145]:
df_test.head()

,n1,n2,edge,rating
0,3896,1098,0,0
1,844,748,0,0
2,3788,10604,0,0
3,9976,844,0,0
4,1366,11356,0,0


In [146]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test.edge == df_test.rating])

# accuracy
right_predictions/len(df_test)

0.9890393320919392

In [162]:
recommend_users(0, G, g)

[('catturd2', 0.11970274),
 ('RealMattCouch', 0.053184852),
 ('charliekirk11', 0.03667799),
 ('JennaEllisEsq', 0.036062352),
 ('LisaMarieBoothe', 0.021839175),
 ('Rothbard1776', 0.017269455),
 ('DineshDSouza', 0.017257238),
 ('RudyGiuliani', 0.01571245),
 ('w_terrence', 0.013755649),
 ('MichaelCoudrey', 0.01252678)]

Adamic-Adar

In [363]:
def get_recommendation_AA(username: str, g:igraph.Graph) -> pd.DataFrame:
    x_neighbors=set()
    y_neighbors=set()
    usersu=[]
    usersv=[]
    for i in g.es:
        usersu.append(i.tuple[0])
        usersv.append(i.tuple[1])
    
    users_unique_list=set(usersu)|set(usersv)
    adamic_avar_data=pd.DataFrame(columns=[username], index=users_unique_list)
    for user_y in users_unique_list:
        if username!=user_y:
            x_neighbors=set(g.neighbors(username))
            y_neighbors=set(g.neighborhood(user_y))
        same_neighbors=x_neighbors&y_neighbors
        aa_val=0
        for n in same_neighbors:
            num_neighbors=len(g.neighbors(n))
            aa_val+=(1/math.log(num_neighbors,10))
        adamic_avar_data[username][user_y]=aa_val
    top_n_recommendations_aa=adamic_avar_data[username].sort_values(ascending=False)
    aa_final_recommendation=pd.DataFrame(top_n_recommendations_aa)
    return aa_final_recommendation.head(10)

In [364]:
user_id=0
get_recommendation_AA(user_id, g)

,0
1282,1.55505
5988,0.960253
516,0.960253
3422,0.960253
10358,0.960253
7290,0.960253
2029,0.960253
4130,0.960253
10700,0.960253
8542,0.960253


In [349]:
set(g.neighbors(1))

{0,
 112,
 266,
 482,
 652,
 914,
 1055,
 1282,
 2168,
 2214,
 2504,
 2866,
 3022,
 3180,
 3210,
 3534,
 3744,
 4174,
 4340,
 4754,
 5418,
 6014,
 6534,
 6902,
 7164,
 7276,
 8188,
 8228,
 8322,
 8434,
 8990,
 9014,
 9068,
 9206,
 9300,
 9660,
 9792,
 9954,
 10484,
 10982,
 11224,
 11468,
 11484,
 11742,
 12316,
 12420,
 12458,
 13436}

In [334]:
def get_neighbors(username: str, usersu: list, usersv: list) -> list:
    neighbors=[]
    for i in range(len(usersu)):
        if (username==usersu[i]):
            neighbors.append(usersv[i])
        if (username==usersv[i]):
            neighbors.append(usersu[i])
    return neighbors

def get_recommendation_AA(username: str, usersu: list, usersv: list) -> pd.DataFrame:
    users_unique_list=set(usersu)|set(usersv)
    adamic_avar_data=pd.DataFrame(columns=[username], index=users_unique_list)
    for user_y in users_unique_list:
        if username!=user_y:
            x_neighbors=set(get_neighbors(username, usersu, usersv))
            y_neighbors=set(get_neighbors(user_y, usersu, usersv))
        same_neighbors=x_neighbors&y_neighbors
        aa_val=0
        for n in same_neighbors:
            num_neighbors=len(get_neighbors(n, usersu, usersv))
            aa_val+=(1/math.log(num_neighbors))
        adamic_avar_data[username][user_y]=aa_val
    top_n_recommendations_aa=adamic_avar_data[username].sort_values(ascending=False)
    aa_final_recommendation=pd.DataFrame(top_n_recommendations_aa)
    return aa_final_recommendation.head(10)

In [336]:
#users_u=[names[0] for names in list(g.edges)]
#users_v=[names[1] for names in list(g.edges)]
#Introduce a username to get its friends recommendations
#u='1Krohmer13'
#get_recommendation_AA(u, users_u, users_v)

PageRank

In [332]:
def pagerank_clearer(pagerank_values: list, user_id: int ,g: igraph.Graph) -> list:
    pagerank=[]
    for i in range(len(pagerank_values)):
        if user_id!=i:
            name=g.vs[i]['name']
            val=float(pagerank_values[i])
            pagerank.append((name, val))
            pagerank=sorted(pagerank, key=lambda x: x[1], reverse=True)
    return pagerank[:][0:10]

In [333]:
vertice_id=0
PageRank_values=g.personalized_pagerank(directed=True, reset_vertices=vertice_id)
PageRank_dict=pagerank_clearer(PageRank_values, vertice_id, g)
print(g.vs[0]['name'])
PageRank_dict

Cindylouwho208


[('aaronjcarpenter', 0.14464007260674197),
 ('GOPLeader', 0.10436465506455363),
 ('m_damuy', 0.01411323598099609),
 ('realDonaldTrump', 0.012462920530383611),
 ('kayleighmcenany', 0.011864371180461712),
 ('mhoran1158', 0.011798789754219368),
 ('Madnessauto69', 0.010989788472787218),
 ('EReldeonk', 0.009768140426121097),
 ('hallroland', 0.009089399533786615),
 ('ZieglerDanny', 0.008438049509322554)]

In [168]:
help(g.personalized_pagerank)

Help on built-in function personalized_pagerank:

personalized_pagerank(...) method of igraph.Graph instance
    personalized_pagerank(vertices=None, directed=True, damping=0.85,
            reset=None, reset_vertices=None, weights=None, 
            arpack_options=None, implementation="prpack", niter=1000,
            eps=0.001)
    
    Calculates the personalized PageRank values of a graph.
    
    The personalized PageRank calculation is similar to the PageRank
    calculation, but the random walk is reset to a non-uniform distribution
    over the vertices in every step with probability M{1-damping} instead of a
    uniform distribution.
    
    @param vertices: the indices of the vertices being queried.
      C{None} means all of the vertices.
    @param directed: whether to consider directed paths.
    @param damping: the damping factor.
      M{1-damping} is the PageRank value for vertices with no
      incoming links.
    @param reset: the distribution over the vertices to b

In [ ]:
#Copy list(g.edges) -> con vocabulary transform to numbers

In [ ]:
users_unique_list=list(vocabulary.keys())

A=np.array(g.edges)
for i in range(len(A)):
    for j in range(len(A[i])):
        A[i][j]=vocabulary[A[i][j]]

weights=np.ones(len(A))
shape=len(users_unique_list)
G = sparse.csr_matrix((weights, (A[:,0], A[:,1])), shape=(shape,shape))
pr=pagerank(G, p=0.85, reverse=True) #This will give the values of the users normalized
pr=[i * shape for i in pr]

In [ ]:
color_map=[]
values=[]
for i in range(len(pr)):
    if pr[i]>10:
        name=list(vocabulary.keys())[list(vocabulary.values()).index(i)]
        values.append(name)

In [ ]:
pos=nx.spring_layout(g)
plt.figure(figsize=(15,15))
#nx.draw_networkx(g, pos, node_shape='o', node_size=50, with_labels=False, font_weight='bold', node_colors=[('red' if (str(u)==str(name)) else 'black') for u in g.nodes()])
nx.draw_networkx(g, pos, node_shape='o', node_size=[1000 if (u in values) else 50 for u in g.nodes], with_labels=False, font_weight='bold', node_color=['red' if (u in values) else 'blue' for u in g.nodes])
plt.show()

In [ ]:
help(nx.pagerank(g, alpha=0.85)

In [165]:
#Definitions that were created for ALS
#def adjency_matrix(vocabulary: dict, edges: list) -> csr_matrix:
#    adj_matrix=pd.DataFrame(0, columns=list(vocabulary.values()), index=list(vocabulary.values()))
#    A=np.array(edges)
#    for i in range(len(A)):
#        A[i][0]=vocabulary[A[i][0]]
#        A[i][1]=vocabulary[A[i][1]]
#    for i in range(len(A)):
#        useru=int(A[i][0])
#        userv=int(A[i][1])
#        adj_matrix[useru][userv]=1
#        adj_matrix[userv][useru]=1
#        adj_matrix_sparse=sparse.csr_matrix(adj_matrix.values)
#    return adj_matrix_sparse

#def similar_users(user_id: int, top: int, vocabulary:dict) -> list:
#    ALS_similar_users=model.similar_users(user_id,top)
#    ALS_similar_users_with_names=number_to_username(ALS_similar_users, vocabulary)
#    return ALS_similar_users_with_names

#def recommend_users(user_id: int, G:csr_matrix, top: int, vocabulary:dict) -> list:
#    ALS_recommended_users=model.recommend(user_id, G, top)
#    ALS_recommended_users_with_names=number_to_username(ALS_recommended_users, vocabulary)
#    return ALS_recommended_users_with_names

#def number_to_username(ALS_list: list, vocabulary: dict) -> list:
#    ALS_list_transformed=[]
#    for i in range(len(ALS_list)):
#        name=list(vocabulary.keys())[list(vocabulary.values()).index(ALS_list[i][0])]
#        new_tuple=(name, ALS_list[i][1])
#        ALS_list_transformed.append(new_tuple)
#    return ALS_list_transformed